In [2]:
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import loggingService
import os
import pandas as pd
import requests
from sentence_transformers import SentenceTransformer

load_dotenv()
logger = loggingService.get_logger()


2023-11-09 09:31:36,344 - numexpr.utils - INFO - Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-11-09 09:31:36,344 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [3]:
# Declare a client instance of the Python Elasticsearch library.
elastic_url = os.getenv("ELASTICSEARCH_URL", "http://127.0.0.1:9200")
index_name = os.getenv("INDEX_NAME", 'vector_index')
model_name_embedding = os.getenv("MODEL_NAME_EMBEDDING", 'sentence-transformers/gtr-t5-large')
path = os.getenv("DATA_PATH", 'data')

api_url = f"{elastic_url}/{index_name}/"
client = Elasticsearch(hosts=[elastic_url])
embeddings_model = SentenceTransformer(model_name_embedding)
embeddings = []
pages = []
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 768, chunk_overlap = 20)
documents = []


In [4]:
df = pd.read_csv(f'{path}/listaPlanosTIMPrd.csv', index_col=0)
df.head()

df.drop(['updatedBy', 'createdBy', 'createdAt', 'updatedAt', 'renewalText'], axis='columns', inplace=True)
df = df.fillna(0)
df = df.astype(str)
# df.to_csv('./data/planosTIMClean.csv')
df['text'] = df['plan'] + ' ' + df['description'] + ' ' + df['dataFranchise'] + ' ' + df['voiceFranchise'] + ' ' + df['smsFranchise'] + ' ' + df['socialNetworkFranchise'] + ' ' + df['mecanism'] + ' ' + df['roamingInternacional'] + ' ' + 'Valor: ' + df['planValues'] 

df.head()
df.describe()

loader = DataFrameLoader(df, page_content_column="text")
documents = loader.load_and_split(text_splitter=text_splitter)


In [16]:
# print(documents[0])
df.columns


Index(['commercialCode', 'plan', 'description', 'dataFranchise',
       'voiceFranchise', 'smsFranchise', 'socialNetworkFranchise', 'mecanism',
       'segment', 'upsellDescription', 'planValues', 'status',
       'roamingInternacional', 'text'],
      dtype='object')

In [6]:
def csv_text_splitter(pdf_text) -> str:
  text = getattr(pdf_text, 'page_content', '')
  metadata = getattr(pdf_text, 'metadata', {})
  
  return {'text': text, 'metadata': metadata}

for doc in documents:
  logger.debug(csv_text_splitter(doc))
  pages.append(csv_text_splitter(doc))

In [7]:
print(pages[0])
print(len(pages))

{'text': 'TIM CONTROLE GIGA C PROMO Com o plano, TIM Controle, GIGA C,PROMO . Você tem ligações ilimitadas,para qualquer operadora usando, o 41e ainda pode enviar SMS ilimitado.  E usar os aplicativos: WhatsApp, Messenger e Telegram ,sem descontar nada da sua franquia de dados. Além de ter, 4.5 Giga de internet para usar como quiser. Contando com o seu plano, e com seus pacotes de internet, você tem #totalDados#, no total. E ainda tem disponível #totalDisponivel# de Internet pra usar como quiser! Com o seu plano, você tem ligações ilimitadas para qualquer operadora, usando o 41. Você tem, SMS ilimitado para TIM, e para outras operadoras. Com o seu plano, você tem o aplicativo Waze para usar à vontade.  Além do Messenger, Telegram, e WhatsApp, para enviar e receber', 'metadata': {'commercialCode': 'PLN586', 'plan': 'TIM CONTROLE GIGA C PROMO', 'description': 'Com o plano, TIM Controle, GIGA C,PROMO . Você tem ligações ilimitadas,para qualquer operadora usando, o 41e ainda pode enviar SM

In [8]:
def create_schema():
    schema = {
        "mappings": {
            "properties": {
                "embedding": {
                    "type": "dense_vector",
                    "dims": 1024,
                    "index": True,
                    "similarity": "cosine"
                },
                "commercialCode": {"type": "text"}, 
                "plan": {"type": "text"},
                "description": {"type": "text"},
                "dataFranchise": {"type": "text"},
                "voiceFranchise": {"type": "text"},
                "smsFranchise": {"type": "text"},
                "socialNetworkFranchise": {"type": "text"},
                "mecanism": {"type": "text"},
                "segment": {"type": "text"},
                "upsellDescription": {"type": "text"},
                "planValues": {"type": "text"},
                "status": {"type": "text"},
                "roamingInternacional": {"type": "text"},
                "text": {"type": "text"}
            }
        }
    }

    response = requests.put(api_url, json=schema)

    logger.info(response.json())
    logger.info(response.status_code)
    
def delete_schema():
    response = requests.delete(f"{elastic_url}/{index_name}/")

    logger.info(response.json())
    logger.info(response.status_code)


In [9]:
def get_embedding(sentence: str,):
  """_summary_

  Args:
      sentence (str): texto para gerar os embeddings

  Returns:
      _type_: List[Tensor] | ndarray | Tensor
  """
  embeddings = embeddings_model.encode(sentence)
  
  return embeddings


In [10]:
def load_data():
    BATCH_SIZE = 1000

    documents_to_index = []
    index = 0

    for document in pages:
        logger.info(f"importing question: {index+1}")
    
        document = {
            "_id": str(index),
            "text": document["text"],
            "embedding": get_embedding(document["text"]),
            "commercialCode": document["metadata"]["commercialCode"],
            "plan": document["metadata"]["plan"],
            "description": document["metadata"]["description"],
            "dataFranchise": document["metadata"]["dataFranchise"],
            "voiceFranchise": document["metadata"]["voiceFranchise"],
            "smsFranchise": document["metadata"]["smsFranchise"],
            "socialNetworkFranchise": document["metadata"]["socialNetworkFranchise"],
            "mecanism": document["metadata"]["mecanism"],
            "segment": document["metadata"]["segment"],
            "upsellDescription": document["metadata"]["upsellDescription"],
            "planValues": document["metadata"]["planValues"],
            "status": document["metadata"]["status"],
            "roamingInternacional": document["metadata"]["roamingInternacional"],
        }
        
        documents_to_index.append(document)
        index += 1

        # To index batches of documents at a time.
        if index % BATCH_SIZE == 0 and index != 0:
            # How you'd index data to Elastic.
            indexing = bulk(client, documents_to_index, index=index_name)
            documents_to_index = []
            logger.info("Success - %s , Failed - %s" % (indexing[0], len(indexing[1])))
        
        # To index the rest, when 'documents' list < BATCH_SIZE.
        if documents_to_index:
            bulk(client, documents_to_index, index=index_name)
        
        logger.info("Finished")
        


In [11]:
# create_schema()


In [18]:
# if __name__ == "__main__":
  # load_data()
query = {
  "field": "embedding",
  "query_vector": get_embedding("quais são as vantagens do plano TIM CONTROLE GIGA C PROMO"),
  "k": 4,
  "num_candidates": 10,
}
source = ['commercialCode', 'plan', 'description', 'dataFranchise',
       'voiceFranchise', 'smsFranchise', 'socialNetworkFranchise', 'mecanism',
       'segment', 'upsellDescription', 'planValues', 'status',
       'roamingInternacional', 'text']

resp = client.search(index=index_name, knn=query, source=source)
print("Got %d Hits:" % resp["hits"]["total"]["value"])
for hit in resp["hits"]["hits"]:
    # print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])
    # print(hit.get('total'))
    print(hit.get("_source").get("text"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Got 4 Hits:
Pré da Tim C Promoção, Pré da TIM, C. sua recarga é transformada automaticamente em benefícios, como dados, Ligações, SMS, Whatsapp, e Messenger, ilimitado. A validade, e o tamanho do pacote de dados, dependem do valor da sua recarga. Olha, você tem disponível #totalDisponivel#, contando com os seus benefícios e bônus, para utilizar como quiser!\nLembrando que, o seu saldo pode ser alterado de acordo com o seu consumo. com a validade dos seus bônus. e com a validade dos seus benefícios. Você pode acompanhar o consumo detalhado, no aplicativo, Meu Tim. Durante o período de validade dos seus benefícios, você pode fazer ligações ilimitadas para qualquer operadora, usando o 41. Nessa promoção, você tem, SMS ilimitado para TIM, ou qualquer outra operadora,
TIM, Pré Tópe, Classic 3.0 TIM, Pré Tópe clássic, 3.0! Nela, sua recarga é transformada automaticamente em benefícios. Você tem: assinatura do Prime Video versão celular, vantagens exclusivas no TIM, Mais Vantagens, além de me